<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/%E5%B0%9D%E8%AF%9515%E6%9C%AC%E5%B0%8F%E8%AF%B4_%E5%AF%B9%E8%AF%9D_%E5%8A%A8%E4%BD%9C%E6%8A%BD%E5%8F%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q zhipuai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 re

In [2]:
import os



import zhipuai

zhipuai.api_key  = "7504"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls /content/drive/MyDrive/Wuxia/input/all_direct

dialogueful_novel.jsonl  part_10.txt  part_2.txt  part_4.txt  part_6.txt  part_8.txt
part_0.txt		 part_1.txt   part_3.txt  part_5.txt  part_7.txt  part_9.txt


In [5]:
# def count_quotes_super(text):

#     quote_chars = '''「」"“”:：'''
#     count = 0
#     for char in text:
#         if char in quote_chars:
#             count += 1
#     return count

def count_quotes_super(text):
    quote_chars = set('「」"“”:：')
    return sum(char in quote_chars for char in text)


取出目标的小说

In [6]:
target_novels = [   "凡人修仙传",   "斗破苍穹",   "仙逆",   "遮天",   "吞噬星空",   "我的美女总裁老婆",   "全职高手",   "傲世九重天",   "网游之永生",   "超级兵王",   "斗罗大陆Ⅱ绝世唐门",   "坏蛋是怎样炼成的Ⅱ",   "修魂记",   "极品神医",   "大周皇族"]
novel2chapters = {}

for novel in target_novels:
    novel2chapters[novel] = []

In [7]:
from tqdm import tqdm


import json



for part_id in tqdm(range(0, 10)):
    fname = "/content/drive/MyDrive/Wuxia/input/all_direct/part_"+ str(part_id) +".txt"

    with open(fname, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            source = data["source"].strip()
            if source not in target_novels:
                continue

            novel2chapters[source].append(data["text"])


100%|██████████| 10/10 [01:57<00:00, 11.70s/it]


In [8]:
for novel in target_novels:
    print(novel, len(novel2chapters[novel]))

凡人修仙传 2477
斗破苍穹 1657
仙逆 2132
遮天 1856
吞噬星空 1694
我的美女总裁老婆 1675
全职高手 1835
傲世九重天 2885
网游之永生 1732
超级兵王 1712
斗罗大陆Ⅱ绝世唐门 1724
坏蛋是怎样炼成的Ⅱ 1491
修魂记 1472
极品神医 1231
大周皇族 1212


这里我们要预处理每个chunk，保留最多quote的部分

In [9]:
!pip install -q tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [10]:
import tiktoken
enc= tiktoken.encoding_for_model("gpt-3.5-turbo")

In [11]:
current_novel = "斗破苍穹"
current_paragraph_id = 0

text = novel2chapters[current_novel][current_paragraph_id]



MAX_TOKEN = 1000

def find_optimal_subsequence(quotes, tokens):
    n = len(tokens)
    total_tokens = sum(tokens)
    max_quotes = 0
    optimal_start_id = 0
    optimal_end_id = 0

    for start_id in range(n):
        tokens_sum = 0
        for end_id in range(start_id, n):
            tokens_sum += tokens[end_id]
            # 当当前子序列的tokens总数小于等于1000时，检查quotes的总和
            if tokens_sum <= MAX_TOKEN:
                current_quotes = sum(quotes[start_id:end_id + 1])
                if current_quotes > max_quotes:
                    max_quotes = current_quotes
                    optimal_start_id = start_id
                    optimal_end_id = end_id
            else:
                # 一旦tokens总数超过1000，跳出内部循环
                break

    return optimal_start_id, optimal_end_id+1, max_quotes

def find_max_quote_part( text ):

    lines = text.split("\n")
    quote_per_line = [count_quotes_super(line) for line in lines]
    # print(quote_per_line)
    token_per_line = [len(enc.encode(line)) for line in lines]
    # print(token_per_line)
    # 执行函数并打印结果
    start_id, end_id,max_quotes = find_optimal_subsequence(quote_per_line, token_per_line)

    current_token = sum(token_per_line[start_id:end_id])

    for new_start_id in range(start_id-1, 0 , -1):
        if current_token + token_per_line[ new_start_id ] <= MAX_TOKEN:
            start_id = new_start_id
            current_token += token_per_line[ new_start_id ]
        else:
            break

    for new_end_id in range(end_id, len(lines)):
        if current_token + token_per_line[new_end_id] <= MAX_TOKEN:
            end_id += 1
            current_token += token_per_line[ new_end_id ]
        else:
            break

    # try naive way
    naive_count = 0
    naive_quote_count = 0
    naive_end = 0
    for i in range(0, len(lines)):
        naive_count += token_per_line[i]
        if naive_count > MAX_TOKEN:
            naive_end = i
            break
        naive_quote_count += quote_per_line[i]

    if_naive = False

    if naive_quote_count * 2 > max_quotes:
        start_id = 0
        end_id = naive_end
        if_naive = True

    rest_text = "\n".join(lines[end_id:])

    return "\n".join( lines[start_id: end_id] ), if_naive, rest_text


print(find_max_quote_part(text)[0])




“斗之力，三段！”
望着测验魔石碑上面闪亮得甚至有些刺眼的五个大字，少年面无表情，唇角有着一抹自嘲，紧握的手掌，因为大力，而导致略微尖锐的指甲深深的刺进了掌心之中，带来一阵阵钻心的疼痛……
“萧炎，斗之力，三段！级别：低级！”测验魔石碑之旁，一位中年男子，看了一眼碑上所显示出来的信息，语气漠然的将之公布了出来……
中年男子话刚刚脱口，便是不出意外的在人头汹涌的广场上带起了一阵嘲讽的骚动。
“三段？嘿嘿，果然不出我所料，这个“天才”这一年又是在原地踏步！”
“哎，这废物真是把家族的脸都给丢光了。”
“要不是族长是他的父亲，这种废物，早就被驱赶出家族，任其自生自灭了，哪还有机会待在家族中白吃白喝。”
“唉，昔年那名闻乌坦城的天才少年，如今怎么落魄成这般模样了啊？”
“谁知道呢，或许做了什么亏心事，惹得神灵降怒了吧……”
周围传来的不屑嘲笑以及惋惜轻叹，落在那如木桩待在原地的少年耳中，恍如一根根利刺狠狠的扎在心脏一般，让得少年呼吸微微急促。
少年缓缓抬起头来，露出一张有些清秀的稚嫩脸庞，漆黑的眸子木然的在周围那些嘲讽的同龄人身上扫过，少年嘴角的自嘲，似乎变得更加苦涩了。
“这些人，都如此刻薄势力吗？或许是因为三年前他们曾经在自己面前露出过最谦卑的笑容，所以，如今想要讨还回去吧……”苦涩的一笑，萧炎落寞的转身，安静地回到了队伍的最后一排，孤单的身影，与周围的世界，有些格格不入。
“下一个，萧媚！”
听着测验人的喊声，一名少女快速的人群中跑出，少女刚刚出场，附近的议论声便是小了许多，一双双略微火热的目光，牢牢的锁定着少女的脸颊……
少女年龄不过十四左右，虽然并算不上绝色，不过那张稚气未脱的小脸，却是蕴含着淡淡的妩媚，清纯与妩媚，矛盾的集合，让得她成功的成为了全场瞩目的焦点……


开始准备数据

In [12]:
query_datas = []
for novel in tqdm(target_novels):
    chapters = novel2chapters[novel]
    for i, chap in enumerate( chapters ):
        rest_text = chap
        for j in range(3):
            current_chunk, if_naive, rest_text = find_max_quote_part( rest_text )
            part_id = novel + "_" + str(i) + "_" + str(j)
            query_datas.append({
                "id":part_id,
                "text":current_chunk
            })
            if not if_naive:
                break
    # break

100%|██████████| 15/15 [02:09<00:00,  8.66s/it]


In [13]:
print(len(query_datas))

52432


In [14]:
query_datas = [data for data in query_datas if len(data["text"]) > 300]

米唯实直接看这里

这个pkl在

https://drive.google.com/file/d/1addQ9H-qZbkqMfaTjN-2c0XyN7wqd-mE/view?usp=sharing

In [15]:
# save query_datas into /content/query_datas.pkl

import pickle

# 将数据保存到文件
with open('/content/drive/MyDrive/CardBuild/exp0204/all_inputs.pkl', 'wb') as file:
    pickle.dump(query_datas, file)


49198

In [16]:
print(len(query_datas))

49198


In [ ]:
!rm -rf /content/output

In [ ]:
print(query_datas[1000]["text"])

“此事不用急，最好还是在这次大战后，再去办这事。毕竟只有让妙音门知道了我们逆星盟的势大，才更好下手一些。到时若是还不肯屈服的话，我听说妙音门有一段时间将总坛设在天星城内，到时我们随便找个私通星宫的借口，还怕此女不乖乖的就范。”老者神色平常的缓缓道。
“好，就这么办。对方只有两名结丹初期的长老，还敢螳臂挡轮不成，还是苍道友足智多谋啊！”壮汉哈哈大笑的大喜道。
老者闻言微然一笑，同时心里有几分自得的暗想道：“这样一来，就可以借势强压妙音门的这位姓韩长老，到时让其将聚集煞气的诀窍一同乖乖的奉上。真是一箭双雕啊！”
想到这里，老者微眯起了双目，脸上露出了一丝诡异的神色。
韩立飞离绿袍老者几人，遁光转眼间就到了港口上空。
看了那些忙碌的逆星盟修士几眼后，他就要加速离开此地。
可就在这时，一股强大的神识毫无遮掩忽然从天而降，一下将韩立罩在了其中。韩立木然一惊，但随即保持神色平静的浮在空中，身形一动不动。
就像他心中早就有所预料的一样，对他们这些结丹期修士，逆星盟怎么可能就这样简单的仅凭件信物就放手。
估计这是那领队的元婴期老怪，亲自用神识来探测一二的。以防有结丹期修士用秘术变幻了容貌，好伪造他人的身份蒙混过关。
韩立的神情镇定异常，没露一丝慌乱之色。他自始至终就未曾想过，用什么假身份来做掩护。因为改变容貌的小把戏，除非两三种传闻中的古籍秘术和几件珍惜之极的宝物外，其余的根本瞒不过神识强大之人的看破。
若是改头换面用什么假身份，万一被人看穿了，反而更显得他心虚可疑，大有弄巧成拙的可能。
如此一来，他宁愿冒着以后被极阴等人追踪的可能，也一直用那妙音门长老的身份一路应付过来。
想起来那些老怪追踪而来的时候，他早已到了外星海了才对。那时就彻底暴露了，也没有什么大的关系了。


In [ ]:
task_prompt = """### Example 1:
input:
5 老师严肃地看着学生们，说：「请大家认真听讲。」
output:
summary:
老师让大家认真听讲
extracted dialogues:
id | [*action*] dialogue | said by | speaker
5 | *严肃地看着学生们* "请大家认真听讲。" | said by | 老师

### Example 2:
input:
2 李晓边走边说：“我们走快点，天快黑了。”张伟回答道：“好的，我跟上你。”
3 “你走慢一点，我有点跟不上了”
4 “加油张伟，部队还在前面等着我们”
output:
summary:
李晓鼓励张伟抓紧赶路
extracted dialogues:
id | [*action*] dialogue | said by | speaker
2 | *边走边说* "我们走快点，天快黑了。" | said by | 李晓
2 | "好的，我跟上你。" | said by | 张伟
3 | "你走慢一点，我有点跟不上了" | said by | 张伟
4 | “加油张伟，部队还在前面等着我们” | said by | 李晓

Please help me convert "input" into a script in CSV format. Extract the dialogues from the paragraphs and output them in CSV format. Let's approach this step by step:
- Firstly, summarize the paragraph, starting with 'summary:'.
- Then, extract dialogues, starting with 'extracted dialogues:'. For each line, if a dialogue occurs, extract it.
- If the speaker performs a certain action, mark it with '*' on both sides of the action.
- Output in the format: id | [*action*] dialogue | said by | speaker.

input:
"""


quote_per_line = [0, 0, 4, 0, 0, 0, 8, 4, 6, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 2, 2, 8, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0]
token_per_line = [98, 63, 148, 98, 68, 77, 121, 45, 67, 40, 132, 144, 103, 96, 100, 33, 69, 154, 76, 116, 65, 76, 74, 41, 114, 72, 112, 43, 103, 56, 122, 36, 42, 101, 66, 104, 68]

我希望实现一个python函数，能够找到最优的start_id和end_id，使得sum(token_per_line[start_id:end_id]) 小于等于1000 的同时 sum(quote_per_line[start_id:end_id] )尽可能大，请用python为我实现

Let's think it step by step

先把问题转化为 左右各删去一个子序列，使得删去的总token > (sum_token - 1000 )的同时，删去的quote之和尽可能小

遍历每一个start_id的情况，来确定最大的end_id

从遍历的每个情况中取出最优的结果


In [ ]:
def add_line_head(text):
    lines = text.split('\n')
    lines = [ line.strip() for line in lines ]
    lines = [ line for line in lines if len(line) > 0 ]
    lines = [ str(id+1) + " " + line for id, line in enumerate(lines)]
    return "\n".join(lines)


In [ ]:
# !pip install -q langchain

# from langchain_community.chat_models import ChatZhipuAI
# from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [ ]:
# async_chat = ChatZhipuAI(
#     temperature=0,
#     api_key=zhipuai.api_key,
#     model="glm-3-turbo",
# )

In [ ]:
# messages = [
#     HumanMessage(content=task_prompt + query_datas[1000]["text"]),
# ]

In [ ]:
messages = [{"role": "user", "content":task_prompt + add_line_head(query_datas[1000]["text"])}]

In [ ]:
from zhipuai import ZhipuAI
client = ZhipuAI(api_key=zhipuai.api_key) # 填写您自己的APIKey
response = client.chat.completions.create(
    model="glm-3-turbo",  # 填写需要调用的模型名称
    messages=messages,
    temperature=0.1
)


In [ ]:
print(response.choices[0].message.content)

summary:
老者计划在大战后利用妙音门的弱势，迫使韩立屈服并获取其秘密。壮汉赞同此计，认为老者足智多谋。老者暗自得意，计划利用形势压迫韩立，并得到其煞气诀窍。韩立飞离老者，但在港口上空被逆星盟修士用神识探测。韩立保持镇定，决定不使用假身份，以免引起怀疑。

extracted dialogues:
id | [*action*] dialogue | said by | speaker
1 | *神色平常的缓缓道* "此事不用急，最好还是在这次大战后，再去办这事。毕竟只有让妙音门知道了我们逆星盟的势大，才更好下手一些。到时若是还不肯屈服的话，我听说妙音门有一段时间将总坛设在天星城内，到时我们随便找个私通星宫的借口，还怕此女不乖乖的就范。" | said by | 老者
2 | *哈哈大笑的大喜道* "好，就这么办。对方只有两名结丹初期的长老，还敢螳臂挡轮不成，还是苍道友足智多谋啊！" | said by | 壮汉
3 | *微然一笑* "这样一来，就可以借势强压妙音门的这位姓韩长老，到时让其将聚集煞气的诀窍一同乖乖的奉上。真是一箭双雕啊！" | said by | 老者
4 | *微眯起了双目，脸上露出了一丝诡异的神色* | said by | 老者
5 | *飞离绿袍老者几人* | said by | 韩立
6 | *看了那些忙碌的逆星盟修士几眼后，就要加速离开此地* | said by | 韩立
7 | *一股强大的神识毫无遮拦忽然从天而降，一下将韩立罩在了其中* | said by | 逆星盟领队的元婴期老怪
8 | *保持神色平静的浮在空中，身形一动不动* | said by | 韩立
9 | *对他们这些结丹期修士，逆星盟怎么可能就这样简单的仅凭件信物就放手* | said by | 韩立
10 | *神识来探测一二的。以防有结丹期修士用秘术变幻了容貌，好伪造他人的身份蒙混过关* | said by | 韩立
11 | *没露一丝慌乱之色。他自始至终就未曾想过，用什么假身份来做掩护* | said by | 韩立
12 | *宁愿冒着以后被极阴等人追踪的可能，也一直用那妙音门长老的身份一路应付过来* | said by | 韩立
13 | *想起来那些老怪追踪而来的时候，他早已到了外星海了才对。那时就彻底暴露了，也没有什么大的关系了。* | said

In [ ]:
async def get_response( messages ):
    client = ZhipuAI(api_key=zhipuai.api_key) # 填写您自己的APIKey
    response = client.chat.completions.create(
        model="glm-3-turbo",  # 填写需要调用的模型名称
        messages=messages,
        temperature=0.1
    )
    return response.choices[0].message.content



In [ ]:
!pip install -q aiofiles

In [ ]:
import os
import json
import time
# import openai
import asyncio
import aiofiles
import tiktoken
import hashlib
# from connector import AsyncPGConnector
from tqdm.asyncio import tqdm as tqdm

enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
en2zh_ratio = 2.3

delay = 1
concurrency_limit = 16

max_file_size = 1024**3

In [ ]:
async def getTranslation(item):
    if "messages" not in item:
        return None
    else:
        messages = item['messages']
        sleep_time = 1
        for i in range(5):
            result = await get_response( messages )
            if result is not None:
                item["response"] = result
                return item
            else:
                await asyncio.sleep(sleep_time)
                sleep_time = min(5, sleep_time * 2)
                continue

    return None

async def process(item, semaphore):
    async with semaphore:
        try:
            output_folder = "/content/output"
            if "output_folder" in item:
                output_folder = item["output_folder"]
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)

            file_path = os.path.join(output_folder, f"{item['id']}.txt")

            if os.path.exists(file_path):
                return

            it = await getTranslation(item)
            if it is None:
                raise Exception(item['id'])

            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump(it, f, ensure_ascii=False)

        except Exception as e:
            # print(f"Error processing entry: {e}")
            return


In [ ]:
def data2messages( data ):
    return [{"role": "user", "content":task_prompt + add_line_head(data["text"])}]

In [ ]:
print(query_datas[0].keys())

dict_keys(['id', 'text'])


In [ ]:
async def main( datas ):
    process_data = []

    output_folder = "/content/output"

    for data in datas:
        process_data.append({
            "id": data['id'],
            "messages": data2messages(data),
            "output_folder": output_folder
        })

    tasks = []

    semaphore = asyncio.Semaphore(concurrency_limit)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for item in process_data:
        file_path = os.path.join(output_folder, f"{item['id']}.txt")

        if os.path.exists(file_path):
            continue

        tasks.append(asyncio.create_task(process(item, semaphore)))

    # # del exist_list
    # print(f"Total items: {len(process_data)}")

    async for task in tqdm(tasks, total=len(tasks), desc="Processing items"):
        await task
        time.sleep(delay)



In [ ]:
!rm -rf /content/output

In [ ]:
datas = query_datas

In [ ]:
!ls /content/drive/MyDrive/CardBuild/exp0204/output/

In [ ]:
start_id = 0
end_id = 10

current_tasks = datas[start_id:end_id]

await main(current_tasks)

temp_output_folder = "/content/output"

for id in range(start_id, end_id):
    id_str = datas[id]["id"]
    file_path = os.path.join(temp_output_folder, f"{id_str}.txt")
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                response = data["response"]
            except:
                continue

        if response is not None:
            datas[id]["response"] = response

final_save_name = "/content/drive/MyDrive/CardBuild/exp0204/output/" + str(start_id) + "_to_" + str(end_id) + ".txt"

with open(final_save_name, 'w', encoding='utf-8') as f:
    for id in range(start_id, end_id):
        json.dump(datas[id], f, ensure_ascii=False)
        f.write('\n')

Processing items: 100%|██████████| 10/10 [02:05<00:00, 12.51s/it]


In [ ]:
super_start = 10
super_end = len(datas)
n_bag = 300

# 计算每个子区间的长度
interval_length = (super_end - super_start) // n_bag

# 生成元组列表
se_tuples_list = [(super_start + i * interval_length, super_start + (i + 1) * interval_length) for i in range(n_bag - 1)]

# 添加最后一个元组，确保最后一个end等于super_end
se_tuples_list.append((se_tuples_list[-1][1], super_end))

# 打印生成的元组列表
print(se_tuples_list[0:2])
print(se_tuples_list[-2:])

import random

random.shuffle(se_tuples_list)


[(10, 173), (173, 336)]
[(48584, 48747), (48747, 49198)]


In [ ]:
for start_id, end_id in se_tuples_list:
    current_tasks = datas[start_id:end_id]

    final_save_name = "/content/drive/MyDrive/CardBuild/exp0204/output/" + str(start_id) + "_to_" + str(end_id) + ".txt"

    if os.path.exists(final_save_name):
        print("skip ", str(start_id) + "_to_" + str(end_id) )
        continue

    await main(current_tasks)
    await main(current_tasks)

    temp_output_folder = "/content/output"

    for id in range(start_id, end_id):
        id_str = datas[id]["id"]
        file_path = os.path.join(temp_output_folder, f"{id_str}.txt")
        if os.path.exists(file_path):
            with open(file_path, 'r', encoding='utf-8') as f:
                try:
                    data = json.load(f)
                    response = data["response"]
                except:
                    continue

            if response is not None:
                datas[id]["response"] = response
        # break

    with open(final_save_name, 'w', encoding='utf-8') as f:
        for id in range(start_id, end_id):
            json.dump(datas[id], f, ensure_ascii=False)
            f.write('\n')



Processing items:   0%|          | 0/163 [00:00<?, ?it/s]